In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os

from sed import SedProcessor
from sed.dataset import dataset
import numpy as np

%matplotlib widget
import matplotlib.pyplot as plt

import json

### for automatic peak finding
from scipy.signal import find_peaks

from sed.calibrator.energy import tof2ev

## Get data paths

If it is your beamtime, you can read the raw data and write to the processed directory. For the public data, you can not write to the processed directory.

The paths are such that if you are on Maxwell, it uses those. Otherwise, data is downloaded in the current directory from Zenodo:
https://zenodo.org/records/12609441

In [ ]:
beamtime_dir = "/asap3/flash/gpfs/pg2/2023/data/11019101" # on Maxwell
if os.path.exists(beamtime_dir) and os.access(beamtime_dir, os.R_OK):
    path = beamtime_dir + "/raw/hdf/offline/fl1user3"
    buffer_path = beamtime_dir + "/processed/tutorial/"
else:
    # data_path can be defined and used to store the data in a specific location
    dataset.get("W110") # Put in Path to a storage of at least 10 Gbyte free space.
    path = dataset.dir
    buffer_path = path + "/processed/"

## Config setup

Here, we get the path to the config file and set up the relevant directories. This can also be done directly in the config file.

In [ ]:
# pick the default configuration file for hextof@FLASH
config_file = Path('../sed/config/flash_example_config.yaml')
assert config_file.exists()

In [ ]:
# here we setup a dictionary that will be used to override the path configuration
config_override = {
    "core": {
        "beamtime_id": 11019101,
        "paths": {
            "data_raw_dir": path,
            "data_parquet_dir": buffer_path
        },
    },
}

### Now we can use those parameters and load our trXPS data using additional config file

In [ ]:
run_number = 44498
sp_44498 = SedProcessor(runs=[run_number], config=config_override, system_config=config_file, verbose=True)
sp_44498.add_jitter()

### We can correct delay stage, t0 position and BAM

In [ ]:
sp_44498.add_delay_offset(
    constant=-1448, # this is time zero position determined from side band fit
    flip_delay_axis=True, # invert the direction of the delay axis
    columns=['bam'], # use the bam to offset the values
    weights=[-0.001], # bam is in fs, delay in ps
    preserve_mean=True # preserve the mean of the delay axis to keep t0 position
)

### Vizualize trXPS data bin in the dldTimeSteps and the corrected delay axis to prepare for energy calibration using SB

In [ ]:
axes = ['dldTimeSteps', 'delayStage']
ranges = [[3900,4200], [-1.5,1.5]]
bins = [100,60]
res_corr = sp_44498.compute(bins=bins, axes=axes, ranges=ranges, normalize_to_acquisition_time="delayStage")

fig,ax = plt.subplots(1,2,figsize=(8,3), layout='constrained')
fig.suptitle(f"Run {run_number}: W 4f, side bands")
res_corr.plot(robust=True, ax=ax[0], cmap='terrain')
ax[0].set_title('raw')
bg = res_corr.sel(delayStage=slice(-1.3,-1.0)).mean('delayStage')
(res_corr-bg).plot(robust=True, ax=ax[1])
ax[1].set_title('difference')

### Automaticaly extract number and position of peaks in the ROI around t0

In [ ]:
# binned data
roi = slice(3980, 4160)
delay = slice(-0.5,0.5)
data = res_corr.sel(dldTimeSteps = roi, delayStage=delay).sum('delayStage')
distance = 7
peaks, _ = find_peaks(data, height=None, distance=distance)

p1SB = data[peaks]['dldTimeSteps'][0]
W4f5 = data[peaks]['dldTimeSteps'][1]
m1SB = data[peaks]['dldTimeSteps'][2]
W4f7 = data[peaks]['dldTimeSteps'][3]
mm1SB = data[peaks]['dldTimeSteps'][4]
plt.figure()
data.plot()
plt.scatter(data[peaks]['dldTimeSteps'], data[peaks], c='r')#, "x")
plt.vlines([p1SB-7,p1SB+7], 0, 150, color='violet', linestyles='dashed', label='$1^{st}$ order SB')
plt.vlines([W4f5-7,W4f5+7], 0, 150, color='b', linestyles='dashed', label='W 4f 7/2')
plt.vlines([m1SB-7,m1SB+7], 0, 150, color='g', linestyles='dashed', label='$-1^{st}$ order SB')
plt.vlines([W4f7-7,W4f7+7], 0, 150, color='r', linestyles='dashed', label='W 4f 5/2')
plt.vlines([mm1SB-7,mm1SB+7], 0, 150, color='orange', linestyles='dashed', label='$2nd -1^{st}$ order SB')
plt.legend()
plt.show()

### find calibration parameters
We now will fit the tof-energy relation. This is done using the maxima of a peak in the ToF spectrum and the known kinetic energy of those peaks (kinetic energy of e.g. W4f peaks (-31.4 and -33.6 eV) and their SB of different orders accounting energy of pump beam of 1030 nm = 1.2 eV. The calibration parameters are obtained by fitting the square root relation.

In [ ]:
### Kinetic energy of w4f peaks and their SB
Ekins = np.array([-30.2,-31.4,-32.6,-33.6,-34.8])
reference = -31.4
ref_id = 1

params=sp_44498.ec.calibrate(
    landmarks = data[peaks]['dldTimeSteps'],
    biases = -1*Ekins,
    ref_id=ref_id,
    ref_energy=reference,
    method="lmfit",
    d={'value':1.0,'min': .8, 'max':1.0, 'vary':True},
    t0={'value':5e-7, 'min': 1e-7, 'max': 1e-6, 'vary':True},
    E0={'value': -100., 'min': -200, 'max': 15, 'vary': True},
    verbose=True)

plt.show()

In [ ]:
sp_44498.save_energy_calibration()

## Append energy axis into a data frame, bin and visualize data in the calibrated energy and corrected delay axis 

In [ ]:
sp_44498.append_energy_axis()

In [ ]:
axes = ['energy', 'delayStage']
ranges = [[-37.5,-27.5], [-1.5,1.5]]
bins = [200,60]
res_corr = sp_44498.compute(bins=bins, axes=axes, ranges=ranges, normalize_to_acquisition_time="delayStage")

fig,ax = plt.subplots(1,2,figsize=(8,3), layout='constrained')
fig.suptitle(f"Run {run_number}: W 4f, side bands")
res_corr.plot(robust=True, ax=ax[0], cmap='terrain')
ax[0].set_title('raw')
bg = res_corr.sel(delayStage=slice(-1.3,-1.0)).mean('delayStage')
(res_corr-bg).plot(robust=True, ax=ax[1])
ax[1].set_title('difference')